# Python - Day 3

https://adventofcode.com/2021/day/3

## Data

In [3]:
import numpy as np
import pandas as pd
import sys
from io import StringIO

In [406]:

example = pd.read_table(
    StringIO(
        """00100
    11110
    10110
    10111
    10101
    01111
    00111
    11100
    10000
    11001
    00010
    01010"""
    ),
    sep = "\t",
    header = None,
    names = ["binary_diagnostic"],
    dtype = {"binary_diagnostic" : str}
)

example["binary_diagnostic"] = example["binary_diagnostic"].str.replace("\s", "")


input = pd.read_table("Input/3_1.txt", sep = "\t", header = None, names = ["binary_diagnostic"], dtype = {"binary_diagnostic" : str})

In [23]:
input.head(5)

,binary_diagnostic
0,011011001101
1,001101100111
2,001101011001
3,110111011101
4,110011101010


In [407]:

example

,binary_diagnostic
0,00100
1,11110
2,10110
3,10111
4,10101
5,01111
6,00111
7,11100
8,10000
9,11001


## Part 1

### Function

In [290]:
def power_consumption_data(binary_diag):

    # split strings into cols
    string_len = len(binary_diag["binary_diagnostic"][1])
    split_strings = binary_diag["binary_diagnostic"].str.split("", expand = True)
    split_strings = split_strings.drop(split_strings.columns[[0, -1]], axis = 1)
    binary_diag = pd.concat([binary_diag, split_strings], axis = 1)

    #reshape 
    binary_diag = binary_diag.melt(id_vars = ["binary_diagnostic"], var_name = "variable", value_name = "value")

    # count 0 and 1 for each position 
    binary_diag = binary_diag.groupby(["variable", "value"]).agg(size = ("value", "count")).reset_index()

    # gamma rate and epsilon rate
    binary_diag = binary_diag.groupby(["variable"]) \
        .apply(lambda x: pd.Series({
                "gamma_rate" : x["value"][x["size"] == np.max(x["size"])].tolist()[0],
                "epsilon_rate" : x["value"][x["size"] == np.min(x["size"])].tolist()[0],
            })
        )
    
    # create return df and convert binary to decimal
    binary_diag = pd.DataFrame({
        "rate" : ["gamma", "epsilon"],
        "binary" : ["".join(map(str, binary_diag["gamma_rate"])), "".join(map(str, binary_diag["epsilon_rate"]))]
    })

    binary_diag["decimal"] = [int(binary, base = 2) for binary in binary_diag["binary"]]
    
    return binary_diag

    
def power_consumption(power_consumption_data):
    return power_consumption_data["decimal"][power_consumption_data["rate"] == "gamma"].iloc[0] * power_consumption_data["decimal"][power_consumption_data["rate"] == "epsilon"].iloc[0]

  



### Result

In [341]:
#Example
power_cons_data = power_consumption_data(example)
result = power_consumption(power_cons_data)
print(
    "Example : ", result
)

#Input
power_cons_data = power_consumption_data(input)
result = power_consumption(power_cons_data)
print(
   "Input : ", result
)


Example :  198
Input :  3429254


## Part 2

### Function

In [416]:
def oxygen_generator_rating(binary_diag):

    string_len = len(binary_diag["binary_diagnostic"][1])
    split_strings = binary_diag["binary_diagnostic"].str.split("", expand = True)
    split_strings = split_strings.drop(split_strings.columns[[0, -1]], axis = 1)
    binary_diag = pd.concat([binary_diag, split_strings], axis = 1)

    #iter over each col until only one binary left
    col_names = list(binary_diag.columns)

    n_number = binary_diag.shape[0] #nrow, number of binary codes
    i = 1 #start col, bite number 1 (not 0 which is the code)

    while n_number > 1:

        #count zeros and ones for bit i
        bit_criteria = binary_diag[col_names[i]]
        ones = bit_criteria[bit_criteria == "1"].count()
        zeros = n_number - ones

        #criteria
        if zeros > ones:
            bit_criteria = "0"
        elif ones >= zeros:
            bit_criteria = "1"

        #filter with criteria
        binary_diag = binary_diag[binary_diag[col_names[i]] == bit_criteria]

        n_number = binary_diag.shape[0]
        i += 1
    
    result = pd.DataFrame({
        "rate" : ["oxygen_generator"]
    })
    result["binary"] = pd.Series(binary_diag["binary_diagnostic"].reset_index(drop=True))
    result["decimal"] = [int(binary, base = 2) for binary in result["binary"]]

    return result


def CO2_scrubber_rating(binary_diag):

    string_len = len(binary_diag["binary_diagnostic"][1])
    split_strings = binary_diag["binary_diagnostic"].str.split("", expand = True)
    split_strings = split_strings.drop(split_strings.columns[[0, -1]], axis = 1)
    binary_diag = pd.concat([binary_diag, split_strings], axis = 1)

    #iter over each col until only one binary left
    col_names = list(binary_diag.columns)

    n_number = binary_diag.shape[0] #nrow, number of binary codes
    i = 1 #start col, bite number 1 (not 0 which is the code)

    while n_number > 1:

        #count zeros and ones for bit i
        bit_criteria = binary_diag[col_names[i]]
        ones = bit_criteria[bit_criteria == "1"].count()
        zeros = n_number - ones

        #criteria
        if ones >= zeros:
            bit_criteria = "0"
        elif zeros > ones:
            bit_criteria = "1"

        #filter with criteria
        binary_diag = binary_diag[binary_diag[col_names[i]] == bit_criteria]

        n_number = binary_diag.shape[0]
        i += 1
    
    result = pd.DataFrame({
        "rate" : ["CO2_scrubber"]
    })
    result["binary"] = pd.Series(binary_diag["binary_diagnostic"].reset_index(drop=True))
    result["decimal"] = [int(binary, base = 2) for binary in result["binary"]]

    return result


def life_support_rating(oxygen_generator_data, CO2_scrubber_data):

    result = pd.DataFrame({
        "rate" : "life_support",
        "decimal" : oxygen_generator_data["decimal"] * CO2_scrubber_data["decimal"]
    })
    result["binary"] = [bin(decimal).replace("0b", "") for decimal in result["decimal"]]

    return result

### Result

In [424]:
#Example
CO2_scrubber_data = CO2_scrubber_rating(example)
oxygen_generator_data = oxygen_generator_rating(example)
life_support_data = life_support_rating(oxygen_generator_data, CO2_scrubber_data)

result = life_support_data["decimal"].tolist()[0]

print(
    "Example : ", result
)
#Input
CO2_scrubber_data = CO2_scrubber_rating(input)
oxygen_generator_data = oxygen_generator_rating(input)
life_support_data = life_support_rating(oxygen_generator_data, CO2_scrubber_data)

result = life_support_data["decimal"].tolist()[0]

print(
    "Input : ", result
)


Example :  230
Input :  5410338
